# 1. 폴더내 PDF 파일을 모두 병합

In [20]:
from PyPDF2 import PdfMerger
import os

def merge_pdfs(directory_path, output_filename):
    # 디렉토리 존재 여부 확인
    if not os.path.exists(directory_path):
        print(f"오류: 디렉토리가 존재하지 않습니다: {directory_path}")
        return
    
    # PDF 병합을 위한 객체 생성
    merger = PdfMerger()
    
    # 디렉토리 내의 모든 PDF 파일 찾기
    pdf_files = [f for f in os.listdir(directory_path) if f.endswith('.pdf')]
    
    # PDF 파일 존재 여부 확인
    if not pdf_files:
        print("오류: 지정된 폴더에 PDF 파일이 없습니다.")
        return
    
    try:
        # 각 PDF 파일을 병합
        for pdf in pdf_files:
            file_path = os.path.join(directory_path, pdf)
            merger.append(file_path)
        
        # 병합된 PDF 저장
        with open(output_filename, 'wb') as output_file:
            merger.write(output_file)
        
        print(f"성공: {len(pdf_files)}개의 PDF 파일이 {output_filename}로 병합되었습니다.")
    
    except Exception as e:
        print(f"오류 발생: {str(e)}")
    
    finally:
        merger.close()

# 사용 예시
directory_path = r"../Chatbot_project/RAG paper"  # PDF 파일들이 있는 폴더 경로
output_filename = "merged.pdf"  # 병합된 PDF 파일명
merge_pdfs(directory_path, output_filename)


Multiple definitions in dictionary at byte 0x41fa7 for key /Type
Multiple definitions in dictionary at byte 0x4d794 for key /Type


성공: 6개의 PDF 파일이 merged.pdf로 병합되었습니다.


# 2. PDF 파일을 마크다운 형식으로 변환

In [21]:
from markitdown import MarkItDown
def convert_pdf_to_markdown(pdf_path: str, md_out_path: str):
    try:
        markitdown = MarkItDown()
        result = markitdown.convert(pdf_path)
        
        # text_content 속성 사용
        markdown_text = result.text_content
        
        with open(md_out_path, 'w', encoding='utf-8') as md_file:
            md_file.write(markdown_text)
            
        return "변환 성공"
        
    except Exception as e:
        return f"오류 발생: {str(e)}"

convert_pdf_to_markdown(r'../Chatbot_project/merged.pdf', 'RAG.md')

'변환 성공'

# 3. vector DB 구현(임베딩은 open ai api 사용 )

구현을 선제적으로 확인하기 위해 임시 QA체인을 포함.

In [22]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import os
import openai
from dotenv import load_dotenv

# --------------------------------------
# (1) 환경 변수 설정
# --------------------------------------
# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# --------------------------------------
# (2) 마크다운 파일 로드 및 벡터스토어 생성
# --------------------------------------
def build_vectorstore_from_markdown(md_file_path: str) -> FAISS:
    """
    마크다운 파일을 FAISS 벡터스토어로 변환
    """
    # 마크다운 파일 읽기
    with open(md_file_path, "r", encoding="utf-8") as f:
        markdown_text = f.read()
    
    # 텍스트 분할
    text_splitter = CharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separator="\n"
    )
    docs = text_splitter.split_text(markdown_text)
    
    if not docs:
        raise ValueError("문서가 비어 있습니다. 마크다운 파일을 확인하세요.")
    
    # OpenAI 임베딩 및 벡터스토어 생성
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(docs, embedding=embeddings)
    
    return vectorstore

# --------------------------------------
# (3) QA 체인 생성
# --------------------------------------
def create_qa_chain(vectorstore: FAISS) -> RetrievalQA:
    """
    질의응답 체인 생성
    """
    llm = ChatOpenAI(
        temperature=0,
        model_name="gpt-3.5-turbo"
    )
    
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 3}
        )
    )
    return qa_chain

# --------------------------------------
# (4) 메인 실행 코드
# --------------------------------------
def main():
    # 마크다운 파일 경로
    md_path = r"../Chatbot_project/RAG.md"
    
    # 벡터스토어 생성
    vectorstore = build_vectorstore_from_markdown(md_path)
    
    # 벡터스토어 저장
    vectorstore.save_local('./db/faiss')
    
    # QA 체인 생성
    qa_chain = create_qa_chain(vectorstore)

    # 질문 실행
    query = input("질문을 입력하세요 : ")    
    response = qa_chain.run(query)
    print(f"\n답변: {response}")

if __name__ == "__main__":
    main()




답변: 프로이트(Freud)는 심리학자이자 심리학의 창시자로 알려진 사람입니다. 그는 무의식, 성욕 이론, 정신 분석 등의 이론으로 유명합니다.
